In [1]:
library(GEOquery)
library(limma)

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.min

Welcome to Bioconductor

    

In [2]:
gse <- getGEO("GSE19615",GSEMatrix=TRUE)
gse <- gse[[1]]

#exp data
eset <- exprs(gse)

Found 1 file(s)
GSE19615_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)
See spec(...) for full column specifications.
File stored at: 
C:\Users\BIOINF~1\AppData\Local\Temp\Rtmp2JjBwT/GPL570.soft


In [3]:
#probe id to gene symbol
fset <- fData(gse)

'↓fset colnames↓'
colnames(fset) #"Symbol" or "Gene Symbol" using

symbols <- fset[,"Gene Symbol"]
rownames(eset) <- symbols

'--------------------------------------------------------------------'
pset <- phenoData(gse)
'↓pset colnames↓'
colnames(pset)

[1] "↓fset colnames↓"

[1] "ID"                               "GB_ACC"                          
 [3] "SPOT_ID"                          "Species Scientific Name"         
 [5] "Annotation Date"                  "Sequence Type"                   
 [7] "Sequence Source"                  "Target Description"              
 [9] "Representative Public ID"         "Gene Title"                      
[11] "Gene Symbol"                      "ENTREZ_GENE_ID"                  
[13] "RefSeq Transcript ID"             "Gene Ontology Biological Process"
[15] "Gene Ontology Cellular Component" "Gene Ontology Molecular Function"

[1] "--------------------------------------------------------------------"

[1] "↓pset colnames↓"

[1] "title"                                    
 [2] "geo_accession"                            
 [3] "status"                                   
 [4] "submission_date"                          
 [5] "last_update_date"                         
 [6] "type"                                     
 [7] "channel_count"                            
 [8] "source_name_ch1"                          
 [9] "organism_ch1"                             
[10] "characteristics_ch1"                      
[11] "characteristics_ch1.1"                    
[12] "characteristics_ch1.2"                    
[13] "characteristics_ch1.3"                    
[14] "characteristics_ch1.4"                    
[15] "characteristics_ch1.5"                    
[16] "characteristics_ch1.6"                    
[17] "characteristics_ch1.7"                    
[18] "characteristics_ch1.8"                    
[19] "characteristics_ch1.9"                    
[20] "characteristics_ch1.10"                   
[21] "characteristics_ch1.11"                   
[22] "characteristics_ch1.12"                   
[23] "characteristics_ch1.13"                   
[24] "characteristics_ch1.14"                   
[25] "characteristics_ch1.15"                   
[26] "characteristics_ch1.16"                   
[27] "molecule_ch1"                             
[28] "extract_protocol_ch1"                     
[29] "label_ch1"                                
[30] "label_protocol_ch1"                       
[31] "taxid_ch1"                                
[32] "hyb_protocol"                             
[33] "scan_protocol"                            
[34] "description"                              
[35] "data_processing"                          
[36] "platform_id"                              
[37] "contact_name"                             
[38] "contact_institute"                        
[39] "contact_address"                          
[40] "contact_city"                             
[41] "contact_state"                            
[42] "contact_zip/postal_code"                  
[43] "contact_country"                          
[44] "supplementary_file"                       
[45] "data_row_count"                           
[46] "relation"                                 
[47] "adjuvant chemotherapy:ch1"                
[48] "age:ch1"                                  
[49] "chemo class:ch1"                          
[50] "distant recur (yn):ch1"                   
[51] "distant recurrence free survival (mo):ch1"
[52] "er:ch1"                                   
[53] "grade (modified, bloom, richardson):ch1"  
[54] "her.2:ch1"                                
[55] "histology type:ch1"                       
[56] "hormonal rx:ch1"                          
[57] "lymph nodes:ch1"                          
[58] "patientid:ch1"                            
[59] "pr:ch1"                                   
[60] "time of followup (mo):ch1"                
[61] "tissue:ch1"                               
[62] "tumor recurrence (36mo):ch1"              
[63] "tumor size (cm):ch1"

In [6]:
input <-readline('1. HRT_p / 2. HRT_n ?: ')

eset2 = t(eset)

if (input == 1){
    pset3 = pset[pset$'er:ch1' != 'neg' & pset$'hormonal rx:ch1' != 'unknown' & pset$'hormonal rx:ch1' != 'none']
    eset2 <- eset2[pset$'er:ch1' != 'neg' & pset$'hormonal rx:ch1' != 'unknown' & pset$'hormonal rx:ch1' != 'none',]
}else if (input == 2){
    pset3 = pset[pset$'hormonal rx:ch1' == 'unknown' | pset$'hormonal rx:ch1' == 'none']
    eset2 <- eset2[pset$'hormonal rx:ch1' == 'unknown' | pset$'hormonal rx:ch1' == 'none',]
}

eset2 = t(eset2)

'----------------------------------------'
'↓eset2↓'
dim(eset2)
'----------------------------------------'
'↓pset3↓'
dim(pset3)
'----------------------------------------'

grp <- pset3$'tumor recurrence (36mo):ch1'
design <- model.matrix(~0 + grp)
colnames(design)
'----------------------------------------'
colnames(design) <- c("ctrl","recurrence")
'↓converted colname↓'
colnames(design)

1. HRT_p / 2. HRT_n ?:  2


[1] "----------------------------------------"

[1] "↓eset2↓"

[1] 54675    51

[1] "----------------------------------------"

...

[1] "----------------------------------------"

[1] "grpno"  "grpyes"

[1] "----------------------------------------"

[1] "↓converted colname↓"

[1] "ctrl"       "recurrence"

In [7]:
#fit
fit <- lmFit(eset2,design)
cont <- makeContrasts(recurrence-ctrl,levels=design)
fit.cont <- contrasts.fit(fit,cont)
fit.cont <- eBayes(fit.cont)
res <- topTable(fit.cont,number=Inf)
head(res)

#data output
if (input == 1){
    write.csv(res, file="GSE19615_DEG_HRT_p.csv")
}else if (input == 2){
    write.csv(res, file="GSE19615_DEG_HRT_n.csv")
}

,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
6374,LOC152225,27.12083,50.51394,5.013723,7.103001e-06,0.1621628,-1.771941
20626,TBX1,33.33555,28.52902,4.990119,7.706122e-06,0.1621628,-1.792080
41492,MDGA1,20.41264,41.08473,4.948400,8.897824e-06,0.1621628,-1.827714
37728,MSX1,21.53500,68.14253,4.746904,1.773290e-05,0.2149025,-2.000385
49674,LOC441204,12.90575,21.61829,4.716652,1.965272e-05,0.2149025,-2.026373
10657,TNFAIP1,53.89551,102.03592,4.450486,4.810289e-05,0.3805247,-2.255368
